In [3]:
import math
import cv2
import numpy as np
import pandas as pd
from datetime import datetime
import time
import os
import re
from pathlib import Path
import json
try:
    from ultralytics import YOLO
    YOLO_AVAILABLE = True
except:
    YOLO_AVAILABLE = False
try:
    import easyocr
    EASYOCR_AVAILABLE = True
except:
    EASYOCR_AVAILABLE = False

In [4]:

class CompactLPRSystem:

    def __init__(self, yolo_model_path='yolov8n.pt', use_yolo=True, ocr_engine='easyocr'):
        print("="*100)
        print(" HỆ THỐNG NHẬN DIỆN BIỂN SỐ XE ")
        print("="*100)
        
        # Tham số xử lý ảnh
        self.processing_params = {
            'adaptive_thresh_block_size': 19,
            'adaptive_thresh_weight': 9,
            'clahe_clip_limit': 3.0,
            'clahe_tile_size': (8, 8),
            'bilateral_d': 9,
            'bilateral_sigma_color': 75,
            'bilateral_sigma_space': 75,
            'morph_kernel_size': (3, 3),
            'min_char_area_ratio': 0.005,
            'max_char_area_ratio': 0.15,
            'min_char_aspect': 0.2,
            'max_char_aspect': 0.85,
            'min_char_height': 15,
            'target_plate_height': 110,
            'target_plate_width': 520
        }
        
        # Load YOLOv8
        self.use_yolo = use_yolo and YOLO_AVAILABLE
        if self.use_yolo:
            print("\n⏳ Đang tải YOLOv8...", end=" ")
            try:
                self.yolo_model = YOLO(yolo_model_path)
                self.yolo_confidence = 0.25
                print("✓")
            except Exception as e:
                print(f"✗ ({e})")
                self.use_yolo = False
        else:
            print("\n chưa tải được yolo")
        
        # Load OCR
        print("⏳ Đang tải OCR Engine...", end=" ")
        self.ocr_engine = 'easyocr'

        if EASYOCR_AVAILABLE:
            self.ocr = easyocr.Reader(['en'], gpu=True)
            print("✓ EasyOCR")
        else:
            raise Exception("EasyOCR chưa được cài đặt. ")

        
        print("="*100)
        print("✓ Hệ thống đã sẵn sàng!\n")
        
        self.all_results = []
        self.detected_plates_list = [] 
    
    # Tính toán tất cả metrics
    """"""                                                         """Tính toán tất cả metrics cho ảnh"""
    def calculate_image_metrics(self, image):
        """Tính toán tất cả metrics cho ảnh"""
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
        
        contrast = float(np.std(gray))
        laplacian = cv2.Laplacian(gray, cv2.CV_64F)
        sharpness = float(laplacian.var())
        brightness = float(np.mean(gray))
        
        h, w = gray.shape
        block_size = 10
        noise_estimates = []
        for i in range(0, h - block_size, block_size):
            for j in range(0, w - block_size, block_size):
                block = gray[i:i+block_size, j:j+block_size]
                noise_estimates.append(np.var(block))
        noise_level = float(np.mean(noise_estimates)) if noise_estimates else 0.0
        
        edges = cv2.Canny(gray, 50, 150)
        edge_density = float(np.sum(edges > 0) / (edges.shape[0] * edges.shape[1]) * 100)
        dynamic_range = float(np.max(gray) - np.min(gray))
        
        return {
            'contrast': round(contrast, 2),
            'sharpness': round(sharpness, 2),
            'brightness': round(brightness, 2),
            'noise_level': round(noise_level, 2),
            'edge_density': round(edge_density, 2),
            'dynamic_range': round(dynamic_range, 2)
        }
    
    """"""                                                         """Đánh giá chất lượng xử lý"""
    def evaluate_quality(self, metrics_original, metrics_final):
        """Đánh giá chất lượng xử lý"""
        contrast_imp = metrics_final['contrast'] / metrics_original['contrast'] if metrics_original['contrast'] > 0 else 1
        sharpness_imp = metrics_final['sharpness'] / metrics_original['sharpness'] if metrics_original['sharpness'] > 0 else 1
        score = (contrast_imp + sharpness_imp) / 2
        
        if score >= 1.8:
            grade, text = "⭐⭐⭐⭐⭐", "Xuất sắc"
        elif score >= 1.5:
            grade, text = "⭐⭐⭐⭐", "Rất tốt"
        elif score >= 1.3:
            grade, text = "⭐⭐⭐", "Tốt"
        elif score >= 1.1:
            grade, text = "⭐⭐", "Khá"
        else:
            grade, text = "⭐", "Cần cải thiện"
        
        return {
            'contrast_improvement': contrast_imp,
            'sharpness_improvement': sharpness_imp,
            'score': score,
            'grade': grade,
            'text': text
        }
    
    # XỬ LÝ ẢNH VỚI LOG CHI TIẾT
    """"""                                                         """Xử lý ảnh với logging chi tiết vào file"""
    def preprocess_plate_with_logging(self, plate_img, log_file, save_folder=None, img_name="plate"):
        all_steps = {}
        all_metrics = {}
        log = []
        
        log.append("="*100)
        log.append(f"🔧 XỬ LÝ ẢNH BIỂN SỐ: {img_name}")
        log.append(f"Thời gian: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        log.append("="*100 + "\n")
        
        # STEP 0: Original
        if len(plate_img.shape) == 3:
            gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
        else:
            gray = plate_img.copy()
        
        all_steps['0_original'] = gray.copy()
        metrics_orig = self.calculate_image_metrics(gray)
        all_metrics['0_original'] = metrics_orig
        
        log.append("📸 STEP 0: ORIGINAL IMAGE")
        log.append(f"   Size: {gray.shape[1]}x{gray.shape[0]} pixels")
        log.append(f"   Contrast: {metrics_orig['contrast']}")
        log.append(f"   Sharpness: {metrics_orig['sharpness']}")
        log.append(f"   Brightness: {metrics_orig['brightness']}\n")
        
        # STEP 1: Resize
        height, width = gray.shape
        target_h = self.processing_params['target_plate_height']
        if height < target_h:
            scale = target_h / height
            resized = cv2.resize(gray, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
        else:
            scale = 1.0
            resized = gray.copy()
        
        all_steps['1_resized'] = resized.copy()
        log.append(f"📐 STEP 1: RESIZE - Scale: {scale:.2f}x, Size: {resized.shape[1]}x{resized.shape[0]}\n")
        
        # STEP 2: CLAHE
        clahe = cv2.createCLAHE(
            clipLimit=self.processing_params['clahe_clip_limit'],
            tileGridSize=self.processing_params['clahe_tile_size']
        )
        enhanced = clahe.apply(resized)
        all_steps['2_clahe'] = enhanced.copy()
        metrics_clahe = self.calculate_image_metrics(enhanced)
        all_metrics['2_clahe'] = metrics_clahe
        
        log.append("✨ STEP 2: CLAHE (Contrast Enhancement)")
        log.append(f"   Contrast: {metrics_orig['contrast']} → {metrics_clahe['contrast']}\n")
        
        # STEP 3: Bilateral Filter
        denoised = cv2.bilateralFilter(
            enhanced,
            self.processing_params['bilateral_d'],
            self.processing_params['bilateral_sigma_color'],
            self.processing_params['bilateral_sigma_space']
        )
        all_steps['3_bilateral'] = denoised.copy()
        
        # STEP 4: Sharpening
        kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
        sharpened = cv2.filter2D(denoised, -1, kernel)
        all_steps['4_sharpened'] = sharpened.copy()
        
        # STEP 5: Adaptive Threshold
        thresh = cv2.adaptiveThreshold(
            sharpened, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV,
            self.processing_params['adaptive_thresh_block_size'],
            self.processing_params['adaptive_thresh_weight']
        )
        all_steps['5_threshold'] = thresh.copy()
        
        # STEP 6: Morphology
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, self.processing_params['morph_kernel_size'])
        morphed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
        all_steps['6_morphology'] = morphed.copy()
        metrics_final = self.calculate_image_metrics(morphed)
        
        evaluation = self.evaluate_quality(metrics_orig, metrics_final)
        
        log.append("="*100)
        log.append("📊 TỔNG KẾT:")
        log.append(f"   🏆 ĐÁNH GIÁ: {evaluation['grade']} {evaluation['text']}")
        log.append(f"   📈 ĐIỂM SỐ: {evaluation['score']:.2f}/2.0")
        log.append("="*100 + "\n")
        
        if save_folder:
            os.makedirs(save_folder, exist_ok=True)
            for step_name, img in all_steps.items():
                cv2.imwrite(os.path.join(save_folder, f"{img_name}_{step_name}.jpg"), img)
        
        with open(log_file, 'w', encoding='utf-8') as f:
            f.write('\n'.join(log))
        
        return morphed, all_steps, all_metrics, evaluation
    
    # PHÂN ĐOẠN KÝ TỰ
    """"""                                                         """Phân đoạn ký tự với logging""" 
    def segment_characters_with_logging(self, thresh_image, plate_roi, log_file, img_name="plate"):
        log = []
        log.append("\n" + "="*100)
        log.append(f"✂️  PHÂN ĐOẠN KÝ TỰ: {img_name}")
        log.append("="*100 + "\n")
        
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
        dilated = cv2.dilate(thresh_image, kernel, iterations=1)
        contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        height, width = thresh_image.shape
        plate_area = height * width
        
        valid_boxes = []
        rejected = {'small': 0, 'large': 0, 'aspect': 0, 'short': 0}
        
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            aspect = w / h if h > 0 else 0
            area_ratio = (w * h) / plate_area
            
            if h < self.processing_params['min_char_height']:
                rejected['short'] += 1
                continue
            if area_ratio < self.processing_params['min_char_area_ratio']:
                rejected['small'] += 1
                continue
            if area_ratio > self.processing_params['max_char_area_ratio']:
                rejected['large'] += 1
                continue
            if not (self.processing_params['min_char_aspect'] < aspect < self.processing_params['max_char_aspect']):
                rejected['aspect'] += 1
                continue
            
            valid_boxes.append((x, y, w, h))
        
        original_count = len(valid_boxes)
        valid_boxes = self.remove_overlapping_boxes(valid_boxes)
        removed_overlap = original_count - len(valid_boxes)
        
        first_line, second_line = self.sort_boxes_by_lines(valid_boxes, height)
        
        log.append(f"📊 KẾT QUẢ: {len(valid_boxes)} ký tự hợp lệ")
        log.append(f"   Dòng 1: {len(first_line)}, Dòng 2: {len(second_line)}")
        log.append("="*100 + "\n")
        
        with open(log_file, 'a', encoding='utf-8') as f:
            f.write('\n'.join(log))
        
        total_chars = len(first_line) + len(second_line)
        if total_chars >= 7:
            grade = "⭐⭐⭐⭐⭐ Hoàn hảo"
        elif total_chars >= 5:
            grade = "⭐⭐⭐ Chấp nhận"
        else:
            grade = "⭐⭐ Kém"
        
        stats = {
            'total_contours': len(contours),
            'valid_chars': len(valid_boxes),
            'first_line': len(first_line),
            'second_line': len(second_line),
            'grade': grade
        }
        
        return first_line, second_line, stats
    
    def remove_overlapping_boxes(self, boxes, threshold=0.5):
        if len(boxes) == 0:
            return []
        boxes = sorted(boxes, key=lambda b: b[0])
        filtered = [boxes[0]]
        for box in boxes[1:]:
            x, y, w, h = box
            last_x, last_y, last_w, last_h = filtered[-1]
            x_overlap = max(0, min(x + w, last_x + last_w) - max(x, last_x))
            if x_overlap < w * threshold:
                filtered.append(box)
        return filtered
    
    def sort_boxes_by_lines(self, boxes, height):
        first, second = [], []
        threshold = height / 2.3
        for box in boxes:
            x, y, w, h = box
            if y + h/2 < threshold:
                first.append(box)
            else:
                second.append(box)
        return sorted(first, key=lambda b: b[0]), sorted(second, key=lambda b: b[0])
    
    # DETECTION VÀ OCR
    def detect_plates_yolo(self, image):
        results = self.yolo_model(image, conf=self.yolo_confidence, verbose=False)
        plates = []
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                plate_img = image[y1:y2, x1:x2]
                if plate_img.size > 0:
                    plates.append({
                        'bbox': (x1, y1, x2, y2),
                        'confidence': conf,
                        'image': plate_img
                    })
        return plates
    
    def ocr_recognize_plate(self, plate_img):
        try:
            if self.ocr_engine == 'paddleocr':
                result = self.ocr.ocr(plate_img, cls=True)
                text = ''.join([line[1][0] for line in result[0]]) if result and result[0] else ""
            else:
                result = self.ocr.readtext(plate_img, detail=0)
                text = ''.join(result)
            
            text = re.sub(r'[^A-Z0-9]', '', text.upper())
            
            if len(text) > 4:
                prefix, suffix = text[:-4], text[-4:]
                for old, new in {'O': '0', 'I': '1', 'Z': '2', 'S': '5', 'B': '8', 'G': '6'}.items():
                    suffix = suffix.replace(old, new)
                text = prefix + suffix
            
            text = text.replace('-', '').replace(' ', '')
            if len(text) >= 7:
                for i, c in enumerate(text[:4]):
                    if c.isalpha():
                        letters_end = i + 1
                if 2 <= letters_end <= 3:
                    return text[:letters_end] + '-' + text[letters_end:]
            
            return text
        except:
            return ""
    
    # PROCESS CHÍNH
    def process_single_image(self, img_path, save_output=True, output_folder="output", log_folder="logs"):
        img_name = os.path.basename(img_path)
        base_name = os.path.splitext(img_name)[0]
        
        print(f"\n{'='*100}")
        print(f"🚗 [{img_name}]")
        print(f"{'='*100}")
        
        try:
            img = cv2.imread(img_path)
            if img is None:
                print("❌ Không đọc được ảnh")
                return None
            
            img = cv2.resize(img, (1920, 1080))
            img_display = img.copy()
            time_start = time.time()
            
            print("⏳ Phát hiện biển số...", end=" ")
            time_det = time.time()
            
            if self.use_yolo:
                plates = self.detect_plates_yolo(img)
            else:
                plates = []
            
            det_time = (time.time() - time_det) * 1000
            print(f"✓ {len(plates)} biển ({det_time:.0f}ms)")
            
            results = []
            
            for i, plate_data in enumerate(plates):
                plate_img = plate_data['image']
                bbox = plate_data['bbox']
                conf = plate_data['confidence']
                
                plate_name = f"{base_name}_plate{i+1}"
                os.makedirs(log_folder, exist_ok=True)
                log_file = os.path.join(log_folder, f"{plate_name}.txt")
                plate_folder = os.path.join(output_folder, plate_name) if save_output else None
                
                print(f"\n   🔧 Biển #{i+1} (Confidence: {conf:.2f})")
                
                print(f"      • Xử lý ảnh...", end=" ")
                time_proc = time.time()
                
                final_img, steps, metrics, evaluation = self.preprocess_plate_with_logging(
                    plate_img, log_file, plate_folder, plate_name
                )
                
                proc_time = (time.time() - time_proc) * 1000
                print(f"✓ ({proc_time:.0f}ms)")
                print(f"        └─ {evaluation['grade']} {evaluation['text']}")
                
                print(f"      • Phân đoạn ký tự...", end=" ")
                first, second, seg_stats = self.segment_characters_with_logging(
                    final_img, plate_img, log_file, plate_name
                )
                print(f"✓ {seg_stats['valid_chars']} ký tự")
                
                print(f"      • Nhận diện OCR...", end=" ")
                time_ocr = time.time()
                plate_text = self.ocr_recognize_plate(steps['4_sharpened'])
                ocr_time = (time.time() - time_ocr) * 1000
                
                if plate_text and len(plate_text.replace('-', '')) >= 6:
                    print(f"✓ [{plate_text}] ({ocr_time:.0f}ms)")
                    
                    # Thêm vào danh sách
                    self.detected_plates_list.append({
                        'image': img_name,
                        'plate_number': plate_text,
                        'confidence': conf,
                        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
                    
                    # Vẽ kết quả
                    x1, y1, x2, y2 = bbox
                    cv2.rectangle(img_display, (x1, y1), (x2, y2), (0, 255, 0), 3)
                    label = f"{plate_text} ({conf:.2f})"
                    (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 1.2, 2)
                    cv2.rectangle(img_display, (x1, y1-h-15), (x1+w, y1), (0, 255, 0), -1)
                    cv2.putText(img_display, label, (x1, y1-5),
                               cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 2)
                    
                    # Vẽ boxes
                    enhanced_boxes = steps['4_sharpened'].copy()
                    if len(enhanced_boxes.shape) == 2:
                        enhanced_boxes = cv2.cvtColor(enhanced_boxes, cv2.COLOR_GRAY2BGR)
                    for x, y, w, h in first:
                        cv2.rectangle(enhanced_boxes, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    for x, y, w, h in second:
                        cv2.rectangle(enhanced_boxes, (x, y), (x+w, y+h), (0, 255, 255), 2)
                    
                    if save_output and plate_folder:
                        cv2.imwrite(os.path.join(plate_folder, f"{plate_name}_7_segmented.jpg"), enhanced_boxes)
                    
                    with open(log_file, 'a', encoding='utf-8') as f:
                        f.write(f"\n{'='*100}\n")
                        f.write(f"🔤 NHẬN DIỆN OCR\n")
                        f.write(f"{'='*100}\n")
                        f.write(f"   Kết quả: {plate_text}\n")
                        f.write(f"   Độ tin cậy: {conf:.2f}\n")
                        f.write(f"{'='*100}\n")
                    
                    results.append({
                        'text': plate_text,
                        'confidence': conf,
                        'bbox': bbox,
                        'evaluation': evaluation,
                        'log_file': log_file
                    })
                    
                    print(f"        └─ 💾 Log: {log_file}")
                else:
                    print(f"✗ Không nhận diện được")
            
            total_time = (time.time() - time_start) * 1000
            
            if save_output and results:
                os.makedirs(output_folder, exist_ok=True)
                result_img = os.path.join(output_folder, f"{base_name}_result.jpg")
                cv2.imwrite(result_img, cv2.resize(img_display, None, fx=0.5, fy=0.5))
            
            print(f"\n{'─'*100}")
            print(f"📊 KẾT QUẢ: ", end="")
            if results:
                print(f"✅ Nhận diện {len(results)}/{len(plates)} biển")
                for r in results:
                    print(f"   • {r['text']} - {r['evaluation']['grade']} {r['evaluation']['text']}")
            else:
                print(f"❌ Không nhận diện được")
            
            print(f"⏱️  THỜI GIAN: Detection {det_time:.0f}ms | Total {total_time:.0f}ms")
            print(f"{'='*100}")
            
            result_data = {
                'image': img_name,
                'detected': len(results) > 0,
                'num_plates': len(results),
                'plates': [r['text'] for r in results],
                'confidences': [r['confidence'] for r in results],
                'evaluations': [r['evaluation'] for r in results],
                'detection_time': det_time,
                'total_time': total_time,
                'log_files': [r['log_file'] for r in results]
            }
            
            self.all_results.append(result_data)
            return result_data
            
        except Exception as e:
            print(f"❌ LỖI: {e}")
            import traceback
            traceback.print_exc()
            return None
    
    def process_folder(self, folder_path, save_output=True, output_folder="output", log_folder="logs"):
        if not os.path.exists(folder_path):
            print(f"❌ Không tồn tại: {folder_path}")
            return
        
        exts = ['.jpg', '.jpeg', '.png', '.bmp']
        images = [f for f in os.listdir(folder_path) if os.path.splitext(f.lower())[1] in exts]
        
        if len(images) == 0:
            print(f"❌ Không có ảnh trong: {folder_path}")
            return
        
        print(f"\n{'#'*100}")
        print(f"# 📁 XỬ LÝ THƯ MỤC: {folder_path}")
        print(f"# 📊 Tổng số: {len(images)} ảnh")
        print(f"{'#'*100}")
        
        for i, img_file in enumerate(images, 1):
            print(f"\n[{i}/{len(images)}]", end=" ")
            img_path = os.path.join(folder_path, img_file)
            self.process_single_image(img_path, save_output, output_folder, log_folder)
        
        print(f"\n{'#'*100}")
        print(f"# ✅ HOÀN THÀNH {len(images)} ẢNH")
        print(f"{'#'*100}\n")
        
        self.print_detected_plates_list()
    
    # IN DANH SÁCH BIỂN SỐ
    """"""                                                         """In danh sách biển số đã nhận diện"""
    def print_detected_plates_list(self):
        if not self.detected_plates_list:
            print(f"\n{'='*100}")
            print(f"📋 DANH SÁCH BIỂN SỐ ĐÃ NHẬN DIỆN")
            print(f"{'='*100}")
            print(f"❌ Không có biển số nào được nhận diện")
            print(f"{'='*100}\n")
            return
        
        print(f"\n{'='*100}")
        print(f"📋 DANH SÁCH BIỂN SỐ ĐÃ NHẬN DIỆN ({len(self.detected_plates_list)} biển)")
        print(f"{'='*100}\n")
        
        print(f"{'STT':<5} {'BIỂN SỐ':<15} {'ẢNH':<40} {'ĐỘ TIN CẬY':<15} {'THỜI GIAN':<20}")
        print(f"{'-'*100}")
        
        for i, plate in enumerate(self.detected_plates_list, 1):
            conf_str = f"{plate['confidence']:.2f} ({plate['confidence']*100:.0f}%)"
            print(f"{i:<5} {plate['plate_number']:<15} {plate['image']:<40} {conf_str:<15} {plate['timestamp']:<20}")
        
        print(f"\n{'='*100}")
        print(f"✅ Tổng cộng: {len(self.detected_plates_list)} biển số")
        print(f"{'='*100}\n")
        
        self.save_plates_to_csv()
    
    """"""                                                         """Lưu danh sách biển số ra file CSV """
    def save_plates_to_csv(self, filename="danh_sach_bien_so.csv"):
        
        try:
            if not self.detected_plates_list:
                return

            df = pd.DataFrame(self.detected_plates_list)
            # Đổi tên cột cho thân thiện tiếng Việt
            df.columns = ['Tên Ảnh', 'Biển Số', 'Độ Tin Cậy', 'Thời Gian']
            
            # Lưu file với encoding utf-8-sig để hiển thị tiếng Việt trên Excel
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            
            print(f"💾 Đã lưu danh sách biển số vào file: {filename}")
        except Exception as e:
            print(f"⚠️ Không thể lưu file CSV: {e}")

    # TỔNG KẾT VÀ BÁO CÁO
    """"""                                                         """Tạo báo cáo tổng kết hoạt động"""
    def generate_summary_report(self, output_file="summary_report.json"):
        if not self.all_results:
            return

        total_imgs = len(self.all_results)
        detected_imgs = sum(1 for r in self.all_results if r['detected'])
        total_plates = sum(r['num_plates'] for r in self.all_results)
        
        avg_det_time = np.mean([r['detection_time'] for r in self.all_results])
        avg_total_time = np.mean([r['total_time'] for r in self.all_results])
        
        # Thống kê chất lượng
        evals = []
        for r in self.all_results:
            evals.extend(r['evaluations'])
        
        if evals:
            avg_score = np.mean([e['score'] for e in evals])
            avg_contrast = np.mean([e['contrast_improvement'] for e in evals])
            avg_sharpness = np.mean([e['sharpness_improvement'] for e in evals])
            
            grades = [e['grade'] for e in evals]
            grade_dist = {g: grades.count(g) for g in set(grades)}
        else:
            avg_score = 0
            avg_contrast = 0
            avg_sharpness = 0
            grade_dist = {}

        report = {
            "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "statistics": {
                "total_images": total_imgs,
                "detected_images": detected_imgs,
                "detection_rate": round(detected_imgs/total_imgs*100, 2) if total_imgs else 0,
                "total_plates_found": total_plates
            },
            "performance": {
                "avg_detection_time_ms": round(avg_det_time, 2),
                "avg_total_time_ms": round(avg_total_time, 2),
                "fps": round(1000/avg_total_time, 2) if avg_total_time > 0 else 0
            },
            "quality_analysis": {
                "average_score": round(avg_score, 2),
                "avg_contrast_improvement": round(avg_contrast, 2),
                "avg_sharpness_improvement": round(avg_sharpness, 2),
                "grade_distribution": grade_dist
            }
        }
        
        # In báo cáo ra console
        print(f"\n{'='*100}")
        print(f"📊 BÁO CÁO TỔNG KẾT HỆ THỐNG")
        print(f"{'='*100}\n")
        
        print(f"📈 THỐNG KÊ CƠ BẢN:")
        print(f"   • Tổng ảnh: {total_imgs}")
        print(f"   • Phát hiện được: {detected_imgs} ({report['statistics']['detection_rate']}%)")
        print(f"   • Tổng biển: {total_plates}\n")
        
        print(f"⚡ HIỆU NĂNG:")
        print(f"   • Detection: {report['performance']['avg_detection_time_ms']}ms (avg)")
        print(f"   • Total: {report['performance']['avg_total_time_ms']}ms (avg)")
        print(f"   • FPS: {report['performance']['fps']}\n")
        
        print(f"🖼️  CHẤT LƯỢNG XỬ LÝ:")
        print(f"   • Contrast improvement: {report['quality_analysis']['avg_contrast_improvement']}x")
        print(f"   • Sharpness improvement: {report['quality_analysis']['avg_sharpness_improvement']}x")
        print(f"   • Điểm trung bình: {report['quality_analysis']['average_score']}/2.0\n")
        
        print(f"   Phân bố đánh giá:")
        for k, v in grade_dist.items():
            pct = v/len(evals)*100
            print(f"      • {k}: {v} ({pct:.1f}%)")
            
        print(f"\n{'='*100}")
        
        try:
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(report, f, ensure_ascii=False, indent=4)
            print(f"💾 Đã lưu: {output_file}")
        except:
            pass
            
        print("\n✅ HOÀN THÀNH!")
        print(f"   📂 Ảnh kết quả: output/")
        print(f"   📄 Log chi tiết: logs/")
        print(f"   📊 Báo cáo: {output_file}")

# MAIN EXECUTION
if __name__ == "__main__":
    INPUT_FOLDER = r"C:\Users\LAN ANH\tran lan anh\HOCTAP\XLA_DA\nguyenducmanhnhandienbienso\image"
    system = CompactLPRSystem(
        yolo_model_path='yolov8n.pt', 
        use_yolo=True, 
        ocr_engine='easyocr' 
    )

    if os.path.exists(INPUT_FOLDER):
        system.process_folder(INPUT_FOLDER, save_output=True)
        system.generate_summary_report()
    else:
        print(f"Không thấy thư mục: {INPUT_FOLDER}")



Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


 HỆ THỐNG NHẬN DIỆN BIỂN SỐ XE 

⏳ Đang tải YOLOv8... ✓
⏳ Đang tải OCR Engine... ✓ EasyOCR
✓ Hệ thống đã sẵn sàng!


####################################################################################################
# 📁 XỬ LÝ THƯ MỤC: C:\Users\LAN ANH\tran lan anh\HOCTAP\XLA_DA\nguyenducmanhnhandienbienso\image
# 📊 Tổng số: 116 ảnh
####################################################################################################

[1/116] 
🚗 [1.jpg]
⏳ Phát hiện biển số... ✓ 2 biển (135ms)

   🔧 Biển #1 (Confidence: 0.56)
      • Xử lý ảnh... ✓ (334ms)
        └─ ⭐⭐⭐⭐⭐ Xuất sắc
      • Phân đoạn ký tự... ✓ 8 ký tự
      • Nhận diện OCR... 

c:\Users\LAN ANH\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


✓ [53L-187760] (1905ms)
        └─ 💾 Log: logs\1_plate1.txt

   🔧 Biển #2 (Confidence: 0.32)
      • Xử lý ảnh... ✓ (305ms)
        └─ ⭐⭐⭐⭐⭐ Xuất sắc
      • Phân đoạn ký tự... ✓ 7 ký tự
      • Nhận diện OCR... ✓ [53L-187760] (1725ms)
        └─ 💾 Log: logs\1_plate2.txt

────────────────────────────────────────────────────────────────────────────────────────────────────
📊 KẾT QUẢ: ✅ Nhận diện 2/2 biển
   • 53L-187760 - ⭐⭐⭐⭐⭐ Xuất sắc
   • 53L-187760 - ⭐⭐⭐⭐⭐ Xuất sắc
⏱️  THỜI GIAN: Detection 135ms | Total 4441ms

[2/116] 
🚗 [10.jpg]
⏳ Phát hiện biển số... ✓ 2 biển (41ms)

   🔧 Biển #1 (Confidence: 0.49)
      • Xử lý ảnh... ✓ (43ms)
        └─ ⭐⭐⭐⭐⭐ Xuất sắc
      • Phân đoạn ký tự... ✓ 3 ký tự
      • Nhận diện OCR... ✗ Không nhận diện được

   🔧 Biển #2 (Confidence: 0.32)
      • Xử lý ảnh... ✓ (66ms)
        └─ ⭐⭐⭐⭐⭐ Xuất sắc
      • Phân đoạn ký tự... ✓ 4 ký tự
      • Nhận diện OCR... ✗ Không nhận diện được

─────────────────────────────────────────────────────────────────────────